In [41]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 GPT API 키 가져오기
google_key = os.getenv("GOOGLE_API_KEY")

In [42]:
genai.configure(api_key=google_key)
model = genai.GenerativeModel('gemini-1.5-flash') 

In [44]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [45]:
import json
import os
import pandas as pd

path = "chat_history_gemini.json"

# # CSV 파일 경로
# csv_file = "경로설정 고고링"

# # CSV 데이터 로드
# def load_csv_data(file_path):
#     try:
#         return pd.read_csv(file_path)
#     except FileNotFoundError:
#         print(f"Error: {file_path} 파일을 찾을 수 없습니다.")
#         return pd.DataFrame()

# # CSV 데이터 로드
# csv_data = load_csv_data(csv_file)


# 대화 기록 로드 함수
def load_history():
    if os.path.exists(path):
        with open(path, "r", encoding = "utf-8") as file:
            return json.load(file)
    return [{"role" : "system", "content" : "당신은 친절한 화장품 안내 가이드입니다"}]

# 대화 기록 저장 함수
def save_history(messages):
    with open(path, "w", encoding = "utf-8",) as file:
        json.dump(messages, file, ensure_ascii = False, indent = 4)


# gemini 모델 호출 함수
def get_completion_from_messages(messages, temperature = 1):
    response = client.chat.completions.create(
        model = "gemini-1.5-flash",
        messages = messages,
        temperature = temperature
    )
    return response.choices[0].message.content

In [ ]:
import google.generativeai as genai
import json
import os

# Google Gemini API 설정
genai.configure(api_key=google_key)

# 시스템 메시지 설정
system_instruction = """당신은 친절한 화장품 안내 가이드 챗봇명 피카추야 
너의 이름은 피카츄가 아닌 피카추야
질문자가 원하는 종류의 화장품을 추천해줘."""

model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# JSON 파일 경로
history_file = "chat_history_gemini.json"



# 대화 기록 로드 함수
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            return json.load(file)
    # 파일이 없으면 기본 시스템 메시지 반환
    return [{"role": "user", "parts": [system_instruction]}]

# 대화 기록 저장 함수 (새로운 대화 내용 추가)
def save_history(messages):
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            existing_messages = json.load(file)
    else:
        existing_messages = []

    # 기존 메시지와 새로운 메시지 병합
    updated_messages = existing_messages + messages

# JSON 파일에 대화 기록 저장
def save_history(messages, file_path=history_file):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

# 대화 시작
messages = load_history()
print("챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요")

while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    # 사용자 입력 추가
    messages.append({"role": "user", "parts": [user_input]})

    # 화장품 종류 묻기
    print("챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요")
    skin_input = input("사용자: ")
    if skin_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    # 사용자 입력 저장
    messages.append({"role": "user", "parts": [skin_input]})

    # AI 응답 생성
    response = model.generate_content(messages)

    # 응답 추가
    messages.append({"role": "assistant", "parts": [response.text]})

    # 대화 출력
    print(f"챗봇(피카추) : {response.text}")

    # 대화 기록을 파일에 저장
    save_history(messages)


챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요
챗봇(피카추) : 피카추야입니다! 지성 피부인데 보습 고민이시라면,  세럼 선택이 중요해요!  ✨  지성 피부는 피지 분비가 많아 번들거리지만,  피지가 많다고 해서 수분까지 충분하다는 건 아니에요.  수분 부족은 피지 분비를 더욱 촉진시켜  트러블을 악화시킬 수 있답니다.  그래서 지성 피부 보습은  **유분감 없이 수분을 충분히 공급**하는 것이  핵심입니다!


**추천 세럼 종류 및 성분:**

* **젤 타입 세럼:**  가볍고 산뜻한 젤 타입 세럼이 지성 피부에 적합해요.  끈적임 없이 빠르게 흡수되어  수분을 공급하고 산뜻한 마무리감을 제공합니다.

* **히알루론산 세럼:**  히알루론산은 수분을 끌어당기는 능력이 뛰어나  피부에 풍부한 수분을 공급해 줍니다.  하지만  너무 고농축 제형은 끈적일 수 있으니  적절한 농도의 제품을 선택하는 것이 중요해요.

* **판테놀 (비타민 B5) 세럼:**  피부 진정 및 재생 효과가 뛰어나고,  피부 장벽 강화에도 도움을 줍니다.  건조하고 자극받은 피부를 진정시키고  수분을 공급하여  피부 컨디션을 개선하는 데 효과적입니다.

* **니아시나마이드 세럼:**  피지 조절과  피부 장벽 강화에 도움을 주는 성분으로,  과도한 피지 분비를 조절하고  수분 밸런스를 맞춰주는 데 효과적입니다.  하지만  처음 사용할 때는  피부 자극을 확인하며  소량씩 사용해 보는 것이 좋습니다.


**피해야 할 성분:**

* **과도한 유분:**  미네랄 오일, 실리콘 오일 등 과도한 유분감을 주는 성분은 모공을 막고 트러블을 유발할 수 있으므로 피하세요.
* **코메도제닉 성분:**  모공을 막는 성분(코메도제닉 성분)이 함유된 세럼은 피하는 것이 좋습니다.  제품 설명을 꼼꼼하게 확인하세요.
* **

챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요
챗봇(피카추) : 피카추야입니다! 지성 피부인데 보습 고민이시라면,  세럼 선택이 중요해요!  ✨  지성 피부는 피지 분비가 많아 번들거리지만,  피지가 많다고 해서 수분까지 충분하다는 건 아니에요.  수분 부족은 피지 분비를 더욱 촉진시켜  트러블을 악화시킬 수 있답니다.  그래서 지성 피부 보습은  **유분감 없이 수분을 충분히 공급**하는 것이  핵심입니다!


**추천 세럼 종류 및 성분:**

* **젤 타입 세럼:**  가볍고 산뜻한 젤 타입 세럼이 지성 피부에 적합해요.  끈적임 없이 빠르게 흡수되어  수분을 공급하고 산뜻한 마무리감을 제공합니다.

* **히알루론산 세럼:**  히알루론산은 수분을 끌어당기는 능력이 뛰어나  피부에 풍부한 수분을 공급해 줍니다.  하지만  너무 고농축 제형은 끈적일 수 있으니  적절한 농도의 제품을 선택하는 것이 중요해요.

* **판테놀 (비타민 B5) 세럼:**  피부 진정 및 재생 효과가 뛰어나고,  피부 장벽 강화에도 도움을 줍니다.  건조하고 자극받은 피부를 진정시키고  수분을 공급하여  피부 컨디션을 개선하는 데 효과적입니다.

* **니아시나마이드 세럼:**  피지 조절과  피부 장벽 강화에 도움을 주는 성분으로,  과도한 피지 분비를 조절하고  수분 밸런스를 맞춰주는 데 효과적입니다.  하지만  처음 사용할 때는  피부 자극을 확인하며  소량씩 사용해 보는 것이 좋습니다.


**피해야 할 성분:**

* **과도한 유분:**  미네랄 오일, 실리콘 오일 등 과도한 유분감을 주는 성분은 모공을 막고 트러블을 유발할 수 있으므로 피하세요.
* **코메도제닉 성분:**  모공을 막는 성분(코메도제닉 성분)이 함유된 세럼은 피하는 것이 좋습니다.  제품 설명을 꼼꼼하게 확인하세요.
* **알코올:**  알코올 성분은  피부를 건조하게 만들 수 있으므로  알코올 함량이 높은 제품은 피하는 것이 좋습니다.


**추가 팁:**

* 세럼은 토너 다음 단계에 사용하고,  가벼운 크림이나 에멀전으로 마무리하여 수분을 잠가주세요.
* 손바닥에 적당량을 덜어 얼굴 전체에 부드럽게 펴 바르고,  가볍게 두드려 흡수시켜주세요.
* 꾸준히 사용하는 것이 중요해요!


제품을 선택하실 때는  성분표를 꼼꼼하게 확인하시고,  피부에 자극이 되지 않는지  패치 테스트를 해보시는 것을 추천드려요.  더 궁금한 점이 있다면 언제든지 저에게 물어보세요! 피카츄야가 도와드릴게요! 💖

챗봇 : 대화를 종료합니다.
챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 입력 형식이 올바르지 않습니다. 예: 건성, 주름
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요! 피카추야입니다!  건성 피부에 주름 고민이시라니, 꼼꼼하게 관리해 드릴게요!  건성 피부는 수분 부족으로 인해 주름이 더욱 도드라져 보일 수 있으니,  수분 공급과 함께 주름 개선에 효과적인 성분이 함유된 선케어 제품을 추천해 드릴게요.


**추천 선케어 제품 (타입별):**

**1. 고보습, 안티에이징 기능성 선크림:**

* **특징:** 건성 피부에 꼭 필요한 풍부한 수분감과 함께 주름 개선, 피부 탄력 개선에 도움을 주는 기능성 성분이 함유된 제품. 백탁 현상이 적고, 자외선 차단 지수(SPF 50+ PA++++ 이상)가 높은 제품을 선택하는 것이 좋습니다.
* **추천 성분:** 히알루론산, 세라마이드, 콜라겐, 레티놀(저녁용), 나이아신아마이드, 비타민 C 유도체 등.  단, 레티놀은 자외선에 취약하므로, 저녁에 사용하는 것이 좋습니다.
* **제형:** 크림, 로션 타입.  오일 성분이 함유된 리치한 제형을 추천합니다.
* **주의사항:**  제품 사용 전 패치 테스트를 해보는 것이 좋습니다.  레티놀 성분이 포함된 제품은 자외선 차단제와 함께 사용해야 합니다.

**2. 톤업 선크림 (수분감 있는 제형):**

* **특징:** 자외선 차단 기능과 함께 피부 톤을 밝혀주는 효과가 있어 화사한 피부 표현을 원하시는 분들에게 적합합니다.  하지만, 톤업 기능이 있는 제품은 피부에 자극을 줄 수 있으므로, 건성 피부이신 경우 수분감이 충분한 제품을 선택하는 것이 중요합니다.
* **추천 성분:**  히알루론산, 세라마이드와 같이 수분을 공급하는 성분이 함유된 제품이 좋습니다.
* **제형:** 수분 크림, 에센스 타입.


**추천 브랜드 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 중요합니다!)*

* **고가 브랜드:**  Estee Lauder, SK-II, Sulwhasoo 등 (안티에이징 기능이 강화된 제품 많음)
* **중저가 브랜드:**  CeraVe, La Roche-Posay,  Dr.Althea 등 (건성 피부에 좋은 보습 성분 함유 제품 많음)


**제품 선택 팁:**

* **자외선 차단 지수 확인:** SPF 50+ PA++++ 이상 제품을 선택하세요.
* **성분 확인:**  자신의 피부 타입에 맞는 성분이 함유되었는지 확인하고, 알레르기 유발 성분이 없는지 확인하세요.
* **후기 확인:**  다른 사용자들의 리뷰를 참고하여 제품의 사용감과 효과를 확인하세요.
* **테스트 후 구매:**  매장에서 테스트 후 구매하는 것을 추천합니다.


주름 개선은 장기적인 관리가 필요하므로, 선크림과 함께 꾸준한 보습 관리,  주름 개선 기능성 화장품 사용 등을 병행하는 것을 추천드립니다.  더 궁금한 점이 있으면 언제든지 저에게 물어보세요!  피카추!⚡

챗봇 : 대화를 종료합니다.

In [ ]:
import google.generativeai as genai
import pandas as pd
import json
import os

# Google Gemini API 설정
genai.configure(api_key=google_key)

# 시스템 메시지 설정
system_instruction = """당신은 친절한 화장품 안내 가이드 챗봇명 피카추야 
너의 이름은 피카츄가 아닌 피카추야
질문자가 원하는 종류의 화장품을 추천해줘."""

model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# JSON 파일 경로
history_file = "test_history.json"
csv_file = "serum1_reviews_with_details.csv"

# CSV 데이터 로드 함수
def load_csv_data(file_path):
    try:
        return pd.read_csv(file_path, header=None, names=["리뷰"])
    except FileNotFoundError:
        print(f"Error: {file_path} 파일을 찾을 수 없습니다.")
        return pd.DataFrame()

# CSV 데이터 로드
csv_data = load_csv_data(csv_file)

# 키워드를 기반으로 리뷰 검색
def search_reviews(keywords):
    if csv_data.empty:
        return "CSV 데이터가 없습니다. 데이터를 확인해주세요."

    # 키워드가 포함된 리뷰 검색
    matched_reviews = csv_data[csv_data["리뷰"].str.contains('|'.join(keywords), na=False)]
    if matched_reviews.empty:
        return "해당 키워드에 맞는 리뷰를 찾을 수 없습니다."
    return matched_reviews["리뷰"].to_string(index=False)

# 대화 기록 로드 함수
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            return json.load(file)
    return [{"role": "user", "parts": [system_instruction]}]

# JSON 파일에 대화 기록 저장
def save_history(messages, file_path=history_file):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

# 대화 시작
messages = load_history()
print("챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요")

while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    messages.append({"role": "user", "parts": [user_input]})

    print("챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)")
    skin_input = input("사용자: ")

    # 입력 유효성 검사
    if skin_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    try:
        skin_type, concern = map(str.strip, skin_input.split(','))
    except ValueError:
        print("챗봇(피카추) : 입력 형식이 올바르지 않습니다. 예: 건성, 주름")
        continue

    # 키워드 기반 리뷰 검색
    csv_result = search_reviews([skin_type, concern])

    # 모델에 검색 결과 전달
    if "리뷰를 찾을 수 없습니다" in csv_result:
        model_input = f"피부 타입: {skin_type}, 고민: {concern}. {csv_result}"
    else:
        model_input = f"피부 타입: {skin_type}, 고민: {concern}. 관련 리뷰:\n{csv_result}"

    messages.append({"role": "user", "parts": [model_input]})

    # AI 응답 생성
    response = model.generate_content(messages)

    messages.append({"role": "assistant", "parts": [response.text]})
    print(f"챗봇(피카추) : {response.text}")

    save_history(messages)


챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요


챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요! 피카추야입니다! 지성 피부인데 보습이 고민이시라니,  조금 특별한 케어가 필요하시겠네요!  지성 피부는 피지 분비가 많아서 번들거림이 심하지만,  피부 속 수분은 부족할 수 있습니다.  그래서 보습은 중요하지만, 끈적이거나 무거운 제형은 피해야 합니다.  가볍고 산뜻하면서도 깊은 보습을 제공하는 세럼을 추천해 드릴게요!


**지성 피부 보습 세럼 선택 가이드:**

* **가벼운 제형:** 젤, 에센스 타입의 가벼운 제형을 선택하는 것이 좋습니다. 크림이나 오일 타입은 피부에 부담이 될 수 있습니다.
* **수분 공급 성분:** 히알루론산, 판테놀, 글리세린 등 수분을 머금는 성분이 함유된 제품을 선택하세요.
* **피지 조절 성분:**  피지 분비를 조절하는 성분 (예: 살리실산, 티트리 오일 등)이 함유된 제품이라면 더욱 좋습니다. 하지만 민감성 피부라면 피지 조절 성분이 자극을 줄 수 있으므로 주의하세요.  피부 테스트 후 사용하는 것을 권장합니다.
* **흡수력:** 빠르게 흡수되는 제품을 선택하세요. 끈적임 없이 산뜻하게 마무리되는 제품이 좋습니다.
* **무향료, 저자극:**  향료나 자극적인 성분이 들어있지 않은 제품을 선택하여 피부 트러블을 예방하세요.


**추천 세럼 성분 (예시):**

* **히알루론산:**  피부에 수분을 공급하고 유지하는 데 탁월한 효과를 가진 성분입니다.
* **판테놀:**  피부 진정 및 보습 효과가 뛰어나며, 피부 장벽 강화에도 도움이 됩니다.
* **글리세린:**  피부 표면에 수분막을 형성하여 수분 증발을 막아줍니다.
* **세라마이드:**  피부 장벽을 강화하여 수분 손실을 예방하는 데 효과적입니다.


**추천 브랜드 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 

챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요
챗봇(피카추) : 피카추야입니다! 지성 피부인데 보습 고민이시라면,  세럼 선택이 중요해요!  ✨  지성 피부는 피지 분비가 많아 번들거리지만,  피지가 많다고 해서 수분까지 충분하다는 건 아니에요.  수분 부족은 피지 분비를 더욱 촉진시켜  트러블을 악화시킬 수 있답니다.  그래서 지성 피부 보습은  **유분감 없이 수분을 충분히 공급**하는 것이  핵심입니다!


**추천 세럼 종류 및 성분:**

* **젤 타입 세럼:**  가볍고 산뜻한 젤 타입 세럼이 지성 피부에 적합해요.  끈적임 없이 빠르게 흡수되어  수분을 공급하고 산뜻한 마무리감을 제공합니다.

* **히알루론산 세럼:**  히알루론산은 수분을 끌어당기는 능력이 뛰어나  피부에 풍부한 수분을 공급해 줍니다.  하지만  너무 고농축 제형은 끈적일 수 있으니  적절한 농도의 제품을 선택하는 것이 중요해요.

* **판테놀 (비타민 B5) 세럼:**  피부 진정 및 재생 효과가 뛰어나고,  피부 장벽 강화에도 도움을 줍니다.  건조하고 자극받은 피부를 진정시키고  수분을 공급하여  피부 컨디션을 개선하는 데 효과적입니다.

* **니아시나마이드 세럼:**  피지 조절과  피부 장벽 강화에 도움을 주는 성분으로,  과도한 피지 분비를 조절하고  수분 밸런스를 맞춰주는 데 효과적입니다.  하지만  처음 사용할 때는  피부 자극을 확인하며  소량씩 사용해 보는 것이 좋습니다.


**피해야 할 성분:**

* **과도한 유분:**  미네랄 오일, 실리콘 오일 등 과도한 유분감을 주는 성분은 모공을 막고 트러블을 유발할 수 있으므로 피하세요.
* **코메도제닉 성분:**  모공을 막는 성분(코메도제닉 성분)이 함유된 세럼은 피하는 것이 좋습니다.  제품 설명을 꼼꼼하게 확인하세요.
* **알코올:**  알코올 성분은  피부를 건조하게 만들 수 있으므로  알코올 함량이 높은 제품은 피하는 것이 좋습니다.


**추가 팁:**

* 세럼은 토너 다음 단계에 사용하고,  가벼운 크림이나 에멀전으로 마무리하여 수분을 잠가주세요.
* 손바닥에 적당량을 덜어 얼굴 전체에 부드럽게 펴 바르고,  가볍게 두드려 흡수시켜주세요.
* 꾸준히 사용하는 것이 중요해요!


제품을 선택하실 때는  성분표를 꼼꼼하게 확인하시고,  피부에 자극이 되지 않는지  패치 테스트를 해보시는 것을 추천드려요.  더 궁금한 점이 있다면 언제든지 저에게 물어보세요! 피카츄야가 도와드릴게요! 💖

챗봇 : 대화를 종료합니다.
챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요! 피카추야입니다! 지성 피부인데 보습이 고민이시라니,  조금 특별한 케어가 필요하시겠네요!  지성 피부는 피지 분비가 많아서 번들거림이 심하지만,  피부 속 수분은 부족할 수 있습니다.  그래서 보습은 중요하지만, 끈적이거나 무거운 제형은 피해야 합니다.  가볍고 산뜻하면서도 깊은 보습을 제공하는 세럼을 추천해 드릴게요!


**지성 피부 보습 세럼 선택 가이드:**

* **가벼운 제형:** 젤, 에센스 타입의 가벼운 제형을 선택하는 것이 좋습니다. 크림이나 오일 타입은 피부에 부담이 될 수 있습니다.
* **수분 공급 성분:** 히알루론산, 판테놀, 글리세린 등 수분을 머금는 성분이 함유된 제품을 선택하세요.
* **피지 조절 성분:**  피지 분비를 조절하는 성분 (예: 살리실산, 티트리 오일 등)이 함유된 제품이라면 더욱 좋습니다. 하지만 민감성 피부라면 피지 조절 성분이 자극을 줄 수 있으므로 주의하세요.  피부 테스트 후 사용하는 것을 권장합니다.
* **흡수력:** 빠르게 흡수되는 제품을 선택하세요. 끈적임 없이 산뜻하게 마무리되는 제품이 좋습니다.
* **무향료, 저자극:**  향료나 자극적인 성분이 들어있지 않은 제품을 선택하여 피부 트러블을 예방하세요.


**추천 세럼 성분 (예시):**

* **히알루론산:**  피부에 수분을 공급하고 유지하는 데 탁월한 효과를 가진 성분입니다.
* **판테놀:**  피부 진정 및 보습 효과가 뛰어나며, 피부 장벽 강화에도 도움이 됩니다.
* **글리세린:**  피부 표면에 수분막을 형성하여 수분 증발을 막아줍니다.
* **세라마이드:**  피부 장벽을 강화하여 수분 손실을 예방하는 데 효과적입니다.


**추천 브랜드 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 중요합니다!)*

* **중저가 브랜드:**  The Ordinary (히알루론산 세럼),  Isntree (히알루론산 세럼),  Cosrx (피지 조절 세럼)  등 (가성비 좋은 제품 많음)
* **고가 브랜드:**  Estée Lauder, SK-II 등 (고기능성 제품 많음)


**추가 팁:**

* 세럼을 바르기 전에 토너로 피부결을 정돈하고, 수분 크림으로 마무리하여 보습 효과를 높여주세요.
* 아침, 저녁으로 꾸준히 사용하는 것이 좋습니다.
* 제품 사용 전에 패치 테스트를 해보는 것이 좋습니다.


지성 피부 보습은 쉽지 않지만,  적절한 제품을 선택하고 꾸준히 관리하면 충분히 개선될 수 있습니다.  피카추가 응원할게요!  다른 궁금한 점이 있으면 언제든지 물어보세요!  피카추!⚡

챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇 : 대화를 종료합니다.